# Random Graph

In [ ]:
!pip install networkit
import networkit as nk
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

## Prepare Parameters

In [ ]:
dimensions = np.load("./dimensions.npy")
print("Dimensions",dimensions)
input_layer_size = dimensions[0]
first_layer_size = dimensions[1]
second_layer_size = dimensions[3]
third_layer_size = dimensions[5]

distribution = np.load("./weights_distribution.npy")
print("Distribution",distribution)
mean1= distribution[0]
var1= distribution[1]
mean2 = distribution[2]
var2 = distribution[3]
mean3 = distribution[4]
var3 = distribution[5]

gradient_distribution = np.load("./gradient_distribution.npy")
print("Gradient Distribution",gradient_distribution)
mean1_grad= gradient_distribution[0]
var1_grad= gradient_distribution[1]
mean2_grad = gradient_distribution[2]
var2_grad = gradient_distribution[3]
mean3_grad = gradient_distribution[4]
var3_grad = gradient_distribution[5]

## Creeate Graph

In [ ]:
graph = nk.Graph(input_layer_size + first_layer_size + second_layer_size + third_layer_size, weighted=True, directed=True)

## Generate Weights

In [ ]:
# Add edges based on weights
fc1_weights = np.ndarray(shape=(first_layer_size, input_layer_size))
fc1_gradients = np.ndarray(shape=(first_layer_size, input_layer_size))

# Input to First layer
for i in tqdm(range(input_layer_size)):
    for j in range(first_layer_size):
      weight = np.random.normal(mean1,var1)
      grad = np.random.poisson(mean1_grad)
      fc1_weights[j][i]=weight
      fc1_gradients[j][i]=grad
      graph.addEdge(i, input_layer_size + j, weight)

In [ ]:
fc2_weights = np.ndarray(shape=(second_layer_size,first_layer_size))
fc2_gradients = np.ndarray(shape=(second_layer_size,first_layer_size))

# First layer to second layer
for i in tqdm(range(first_layer_size)):
    for j in range(second_layer_size):
        weight = np.random.normal(mean2, var2)
        grad = np.random.poisson(mean2_grad)
        fc2_weights[j][i]=weight
        fc2_gradients[j][i]=grad
        graph.addEdge(input_layer_size + i, input_layer_size + first_layer_size + j, weight)

In [ ]:
fc3_weights = np.ndarray(shape=(third_layer_size, second_layer_size))
fc3_gradients = np.ndarray(shape=(third_layer_size, second_layer_size))

# Second layer to third layer
for i in tqdm(range(second_layer_size)):
    for j in range(third_layer_size):
        weight = np.random.normal(mean3, var3)
        grad = np.random.poisson(mean3_grad)
        fc3_weights[j][i]=weight
        fc3_gradients[j][i]=grad
        graph.addEdge(input_layer_size + first_layer_size + i, input_layer_size + first_layer_size + second_layer_size + j, weight)

## Check network is similar to real

In [ ]:
# Now you can use the graph object
print("Nodes",graph.numberOfNodes())
print("Edges",graph.numberOfEdges())

In [ ]:
# Assuming fc1_weights is already defined as in your previous code
plt.subplot(1, 3, 1)
plt.hist(fc1_weights.flatten(), bins=3000, range=(-0.10, 0.10))
plt.xlabel("Weight Value")
plt.ylabel("Frequency")
plt.title("FC1")

# Assuming fc2_weights is already defined as in your previous code
plt.subplot(1, 3, 2)
plt.hist(fc2_weights.flatten(), bins=3000, range=(-0.10, 0.10))
plt.xlabel("Weight Value")
plt.ylabel("Frequency")
plt.title("FC2")

# Assuming fc3_weights is already defined as in your previous code
plt.subplot(1, 3, 3)
plt.hist(fc3_weights.flatten(), bins=3000, range=(-0.10, 0.10))
plt.xlabel("Weight Value")
plt.ylabel("Frequency")
plt.title("FC3")

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.9,
                    hspace=0.9)

if not os.path.isdir('./output/'):
    os.makedirs('./output')

plt.savefig("./output/random_weights_distribution.jpg")
plt.show()

In [ ]:
# Assuming fc1_gradients_magnitude is already defined as in your previous code
plt.subplot(1, 3, 1)
plt.hist(fc1_gradients.flatten(), bins=300, range=(0, 0.50))
plt.xlabel("Gradient Value")
plt.ylabel("Frequency")
plt.title("FC1")

# Assuming fc2_gradients_magnitude is already defined as in your previous code
plt.subplot(1, 3, 2)
plt.hist(fc2_gradients.flatten(), bins=200, range=(0, 0.50))
plt.xlabel("Gradient Value")
plt.ylabel("Frequency")
plt.title("FC2")

# Assuming fc1_gradients_magnitude is already defined as in your previous code
plt.subplot(1, 3, 3)
plt.hist(fc3_gradients.flatten(), bins=200, range=(0, 0.50))
plt.xlabel("Gradient Value")
plt.ylabel("Frequency")
plt.title("FC3")

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.9,
                    hspace=0.9)

if not os.path.isdir('./output/'):
    os.makedirs('./output')

plt.savefig("./output/random_gradient_distribution.jpg")
plt.show()

# Compute Embedding Features

## Estimate Weighted Betweenness for nodes

In [ ]:
# Compute approximation of betweenness centrality
betweenness = nk.centrality.EstimateBetweenness(graph, int(graph.numberOfNodes()*.35), normalized=True, parallel=True)

betweenness.run()
betweenness_scores = betweenness.scores()

# Print or further process the betweenness centrality scores
print("Estimated Betweenness Centrality Scores:", betweenness_scores)

if not os.path.isdir('./output/'):
    os.makedirs('./output')
np.save("./output/random_estimated_betweenness.npy", betweenness_scores)

##Weighted Degree

In [ ]:
degrees = []

for i in tqdm(range(input_layer_size + first_layer_size + second_layer_size + third_layer_size)):
  degrees.append((abs(graph.weightedDegree(i))+abs(graph.weightedDegreeIn(i)))/(graph.degreeIn(i)+graph.degreeOut(i)))

print(degrees)

if not os.path.isdir('./output/'):
    os.makedirs('./output')
np.save("./output/random_weighted_degree.npy", degrees)

In [ ]:
norm = np.linalg.norm(degrees)
if norm!=0:
  normalized_degrees = degrees/norm

print(normalized_degrees)

if not os.path.isdir('./output/'):
    os.makedirs('./output')
np.save("./output/random_normalized_degree.npy", normalized_degrees)

##Normalized Gradient

In [ ]:
weighted_node_gradients = []

for i in tqdm(range(input_layer_size)):
    weighted_node_gradients.append(np.sum(np.abs(np.multiply(fc1_gradients[:,i],fc1_weights[:,i]))))

for i in tqdm(range(first_layer_size)):
    weighted_node_gradients.append(np.sum(np.abs(np.multiply(fc2_gradients[:,i],fc2_weights[:,i]))))

for i in tqdm(range(second_layer_size)):
    weighted_node_gradients.append(np.sum(np.abs(np.multiply(fc3_gradients[:,i],fc3_weights[:,i]))))

for i in tqdm(range(third_layer_size)):
    weighted_node_gradients.append(0.0)

print(weighted_node_gradients)

if not os.path.isdir('./output/'):
    os.makedirs('./output')
np.save("./output/random_weighted_gradient.npy", weighted_node_gradients)

In [ ]:
norm = np.linalg.norm(weighted_node_gradients)
if norm!=0:
  normalized_weighted_node_gradients = weighted_node_gradients/norm

print(normalized_weighted_node_gradients)

if not os.path.isdir('./output/'):
    os.makedirs('./output')
np.save("./output/random_normalized_gradient.npy", normalized_weighted_node_gradients)

# **Building graphs for paths computation**

 ## Building network with absolute value of weights for shortest path computation

In [ ]:
abs_graph = nk.Graph(input_layer_size + first_layer_size + second_layer_size + third_layer_size, weighted=True, directed=True)
# FC1
for i in tqdm(range(input_layer_size)):
    for j in range(first_layer_size):
        abs_graph.addEdge(i, input_layer_size + j, abs(fc1_weights[j, i]))

# FC2
for i in tqdm(range(first_layer_size)):
    for j in range(second_layer_size):
        abs_graph.addEdge(input_layer_size + i, input_layer_size + first_layer_size + j, abs(fc2_weights[j,i]))

# FC3
for i in tqdm(range(second_layer_size)):
    for j in range(third_layer_size):
        abs_graph.addEdge(input_layer_size + first_layer_size + i, input_layer_size + first_layer_size+ second_layer_size + j, abs(fc3_weights[j,i]))

## Shortest Paths computation

In [ ]:
shortest_paths_distances = []   # Store distances separately
shortest_paths_nodes = []       # Store paths (nodes) separately

heuristic = [0 for _ in range(abs_graph.upperNodeIdBound())]

for i in tqdm(range(0,input_layer_size)):
    # print("searching for node",i)
    min_distance = float('inf')
    min_path = []
    dijkstra = nk.distance.Dijkstra(abs_graph, source=i, storePaths=True)
    dijkstra.run()

    for j in range(input_layer_size + first_layer_size + second_layer_size, input_layer_size + first_layer_size+ second_layer_size + third_layer_size):
        # print("To node",j)
        if (dijkstra.distance(j)<min_distance):
            min_distance = dijkstra.distance(j)
            min_path = dijkstra.getPath(j)

    # print("[",min_distance,",",min_path,"]")
    shortest_paths_distances.append(min_distance)
    shortest_paths_nodes.append(min_path)

if not os.path.isdir('./output/'):
    os.makedirs('./output')
np.save("./output/random_shortest_paths_distances.npy", shortest_paths_distances)
np.save("./output/random_shortest_paths_nodes.npy", shortest_paths_nodes)

In [ ]:
del(abs_graph)
del(shortest_paths_distances)
del(shortest_paths_nodes)
del(dijkstra)

## Inverting weights to find longest paths

In [ ]:
abs_graph = nk.Graph(input_layer_size + first_layer_size + second_layer_size + third_layer_size, weighted=True, directed=True)

# FC1
for i in tqdm(range(input_layer_size)):
    for j in range(first_layer_size):
        abs_graph.addEdge(i, input_layer_size + j, -abs(fc1_weights[j, i]))

# FC2
for i in tqdm(range(first_layer_size)):
    for j in range(second_layer_size):
        abs_graph.addEdge(input_layer_size + i, input_layer_size + first_layer_size + j, -abs(fc2_weights[j,i]))

# FC3
for i in tqdm(range(second_layer_size)):
    for j in range(third_layer_size):
        abs_graph.addEdge(input_layer_size + first_layer_size + i, input_layer_size + first_layer_size+ second_layer_size + j, -abs(fc3_weights[j,i]))

## Finding longest_paths

In [ ]:
longest_paths_distances = []   # Store distances separately
longest_paths_nodes = []       # Store paths (nodes) separately

heuristic = [0 for _ in range(abs_graph.upperNodeIdBound())]

for i in tqdm(range(0,input_layer_size)):
    # print("searching for node",i)
    min_distance = float('-inf')
    min_path = []
    dijkstra = nk.distance.Dijkstra(abs_graph, source=i, storePaths=True)
    dijkstra.run()

    for j in range(input_layer_size + first_layer_size+ second_layer_size, input_layer_size + first_layer_size+ second_layer_size + third_layer_size):
        # print("To node",j)
        if (dijkstra.distance(j)>min_distance):
            min_distance = dijkstra.distance(j)
            min_path = dijkstra.getPath(j)

    # print("[",-min_distance,",",min_path,"]")
    longest_paths_distances.append(-min_distance)
    longest_paths_nodes.append(min_path)

if not os.path.isdir('./output/'):
    os.makedirs('./output')
np.save("./output/random_longest_paths_distances.npy", longest_paths_distances)
np.save("./output/random_longest_paths_nodes.npy", longest_paths_nodes)

# Compress Output Folder

In [ ]:
!zip -r /content/Random_Results.zip /content/output